In [1]:
import os
#print(os.environ["CUDA_HOME"])
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=''
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10656511755782538421
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6846381658741471928
physical_device_desc: "device: XLA_CPU device"
]


In [2]:
! python --version

Python 3.6.6 :: Anaconda, Inc.


# Feature vector NNs: CNN + multilayer perceptron

In [11]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np

In [12]:
JOINT_VECTOR_FILE = '../metadata/joint_file.csv'
DATA_ORDER_FILE = '../metadata/data_order.csv'

### Load data

In [13]:
joint_vector_df = pd.read_csv(JOINT_VECTOR_FILE, index_col=0)
data_order_df = pd.read_csv(DATA_ORDER_FILE, header=None, index_col=0, usecols=[0])
LAST_TRAIN_INDEX = 1782
# przy indeksowaniu od 0, od indeksu 1782 do końca zaczynają się indeksy testowe. (20% całego zbioru)

In [14]:
print(joint_vector_df.shape)
joint_vector_df.iloc[:5, :]

(2174, 1687)


,5th_percentile_0,5th_percentile_1,5th_percentile_2,5th_percentile_3,5th_percentile_4,5th_percentile_5,5th_percentile_6,5th_percentile_7,5th_percentile_8,5th_percentile_9,...,var_237,var_238,var_239,Loop scattering,Background ring,Strong background,Diffuse scattering,Artifact,Ice ring,Non-uniform detector
auw1_16_1_001,194,188,0,169,167,147,126,87,73,86,...,7,7,8,0,1,0,1,0,0,0
120358_1_E1_001,0,240,237,237,238,238,233,224,208,177,...,40,26,21,0,1,1,1,0,0,0
czx8-9,226,221,214,213,216,215,216,211,211,210,...,53,49,54,1,1,0,0,0,0,0
27025_2_001,251,249,247,249,242,223,199,162,151,135,...,15,15,15,0,1,1,0,0,0,1
zxp4-9,243,236,216,187,161,150,149,146,128,82,...,21,20,19,0,1,0,0,0,0,0


In [15]:
print(data_order_df.shape)
data_order_df.iloc[:5, :]

(2228, 0)


""
0
16-m2b3g-peak.0001
pip-c9ge-1peak.0001
T1570_1_1000
230508b11_z_0001
7303_4_001


In [16]:
train_names_df = data_order_df.iloc[:LAST_TRAIN_INDEX, :]
test_names_df = data_order_df.iloc[LAST_TRAIN_INDEX:, :]
print(f'Training: {train_names_df.shape}')
print(f'Testing: {test_names_df.shape}')

Training: (1782, 0)
Testing: (446, 0)


In [17]:
train_df = train_names_df.join(joint_vector_df, how='inner')
test_df = test_names_df.join(joint_vector_df, how='inner')
print(f'Training: {train_df.shape}')
print(f'Testing: {test_df.shape}')

train_x_df = train_df.iloc[:, :-7]
train_y_df = train_df.iloc[:, -7:]
test_x_df = test_df.iloc[:, :-7]
test_y_df = test_df.iloc[:, -7:]
print(f'Training X: {train_x_df.shape}')
print(f'Training Y: {train_y_df.shape}')
print(f'Testing X: {test_x_df.shape}')
print(f'Testing Y: {test_y_df.shape}')

Training: (1719, 1687)
Testing: (433, 1687)
Training X: (1719, 1680)
Training Y: (1719, 7)
Testing X: (433, 1680)
Testing Y: (433, 7)


In [18]:
train_x_df.iloc[:5, :]

,5th_percentile_0,5th_percentile_1,5th_percentile_2,5th_percentile_3,5th_percentile_4,5th_percentile_5,5th_percentile_6,5th_percentile_7,5th_percentile_8,5th_percentile_9,...,var_230,var_231,var_232,var_233,var_234,var_235,var_236,var_237,var_238,var_239
pip-c9ge-1peak.0001,239,238,230,226,222,216,197,175,174,171,...,8,9,8,8,8,9,8,8,8,8
T1570_1_1000,222,209,174,176,173,179,184,177,184,186,...,24,24,24,25,25,23,25,23,24,24
230508b11_z_0001,253,253,251,250,170,14,16,6,1,0,...,22,18,17,18,20,22,18,17,17,17
95613_1_E2,0,233,223,217,205,197,180,158,158,156,...,64,63,59,58,60,61,62,60,55,59
idp90515_3ULW.0001,0,251,240,229,221,215,212,207,171,136,...,9,9,8,8,8,8,8,7,7,7


In [19]:
train_y_df.iloc[:5, :]

,Loop scattering,Background ring,Strong background,Diffuse scattering,Artifact,Ice ring,Non-uniform detector
pip-c9ge-1peak.0001,0,1,0,1,0,0,0
T1570_1_1000,0,0,0,0,0,0,1
230508b11_z_0001,0,1,1,0,0,0,0
95613_1_E2,0,1,0,0,1,0,1
idp90515_3ULW.0001,0,0,0,0,0,1,0


In [20]:
test_x_df.iloc[:5, :]

,5th_percentile_0,5th_percentile_1,5th_percentile_2,5th_percentile_3,5th_percentile_4,5th_percentile_5,5th_percentile_6,5th_percentile_7,5th_percentile_8,5th_percentile_9,...,var_230,var_231,var_232,var_233,var_234,var_235,var_236,var_237,var_238,var_239
159752_1_E2_0001,0,225,224,219,221,218,214,204,183,168,...,46,49,43,42,42,43,42,46,43,43
140722_1_E1_0001,246,230,235,230,234,232,225,212,195,201,...,25,25,24,25,24,24,24,23,23,22
146092zn_1_E2_0001,229,229,226,224,224,224,226,221,223,221,...,47,49,46,49,46,47,46,47,45,46
192523_1_00001,254,252,249,244,238,218,187,163,115,78,...,39,35,35,36,38,37,35,31,34,41
135840_1_E2_0001,253,250,250,250,250,250,250,246,243,230,...,12,11,12,11,11,11,11,11,10,10


In [21]:
test_y_df.iloc[:5, :]

,Loop scattering,Background ring,Strong background,Diffuse scattering,Artifact,Ice ring,Non-uniform detector
159752_1_E2_0001,0,1,1,0,0,0,0
140722_1_E1_0001,0,1,0,0,0,0,0
146092zn_1_E2_0001,0,1,0,0,0,0,0
192523_1_00001,1,0,1,0,1,0,0
135840_1_E2_0001,0,1,0,1,0,0,1


### Set common model constants

In [22]:
NUM_INPUT = train_x_df.shape[1]
NUM_CLASSES = 2
VECTOR_LENGTH = 240
NUM_FEATURES = 7

### Reshape pandas df to scaled np

In [23]:
train_x_np = train_x_df.values.astype(float)/255
test_x_np = test_x_df.values.astype(float)/255

In [24]:
print(train_x_np)
print(test_x_np)

[[0.9372549  0.93333333 0.90196078 ... 0.03137255 0.03137255 0.03137255]
 [0.87058824 0.81960784 0.68235294 ... 0.09019608 0.09411765 0.09411765]
 [0.99215686 0.99215686 0.98431373 ... 0.06666667 0.06666667 0.06666667]
 ...
 [0.95686275 0.92941176 0.89803922 ... 0.10196078 0.10196078 0.10196078]
 [0.         0.97254902 0.97647059 ... 0.0627451  0.0627451  0.05882353]
 [0.         0.98039216 0.93333333 ... 0.19607843 0.18431373 0.20392157]]
[[0.         0.88235294 0.87843137 ... 0.18039216 0.16862745 0.16862745]
 [0.96470588 0.90196078 0.92156863 ... 0.09019608 0.09019608 0.08627451]
 [0.89803922 0.89803922 0.88627451 ... 0.18431373 0.17647059 0.18039216]
 ...
 [0.94509804 0.90980392 0.91764706 ... 0.17647059 0.17647059 0.16862745]
 [0.96862745 0.96862745 0.9372549  ... 1.         1.         1.        ]
 [0.92156863 0.91372549 0.89411765 ... 0.12156863 0.11372549 0.12156863]]


### Get labels for chosen class

In [25]:
train_y_df.columns

Index(['Loop scattering', 'Background ring', 'Strong background',
       'Diffuse scattering', 'Artifact', 'Ice ring', 'Non-uniform detector'],
      dtype='object')

In [ ]:
FEATURE_NAME = 'Loop scattering'
train_y = train_y_df[FEATURE_NAME].values
test_y = test_y_df[FEATURE_NAME].values

### With *layers* API
#### conv net

In [52]:
# TODO toCaps
learning_rate = 0.001
num_steps = 2000
batch_size = 128
dropout = 0.25

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [53]:
from F_vector_nn_utils import conv_net, model_fn

model = tf.estimator.Estimator(model_fn)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': train_x_np}, y=train_y,
    batch_size=batch_size, num_epochs=None, shuffle=True)

model.train(input_fn, steps=num_steps)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_x_np}, y=test_y,
    batch_size=batch_size, shuffle=False)

e = model.evaluate(input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpa8bingjq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa446d384a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [54]:
print(e)

dict2dump = {}
for k,v in e.items():
    dict2dump[k] = str(v)

import json
RESULT_DIR = '../results/cnn/'
with open(f'{RESULT_DIR}{FEATURE_NAME}.json', 'w') as file:
    json.dump(dict2dump, file)

{'accuracy': 0.9099307, 'f1': 0.91754746, 'loss': 0.47948915, 'precision': 0.93133044, 'recall': 0.90416664, 'roc_auc': 0.91063255, 'global_step': 2000}


### Raw NN - without _layer_ API
#### 2-Hidden Layers Fully Connected NN

In [ ]:
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

n_hidden_1 = 256 # no. neurons - 1st layer
n_hidden_2 = 256 # no. neurons - 2nd layer

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
X = tf.placeholder("float", [None, NUM_INPUT])
Y = tf.placeholder("float", [None, NUM_CLASSES])

weights = {
    'h1': tf.Variable(tf.random_normal([NUM_INPUT, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, NUM_CLASSES]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([NUM_CLASSES]))
}


# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session(config=config) as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))